<a href="https://colab.research.google.com/github/jimitogni/ml_study/blob/master/dogs_breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
/content/drive/My Drive/UNICAMP/databases

In [0]:
!mkdir .kaggle

In [0]:
!ls -lha .kaggle/

total 12K
drwxr-xr-x 2 root root 4.0K May  3 03:44 .
drwxr-xr-x 1 root root 4.0K May  3 03:43 ..
-rw-r--r-- 1 root root   65 May  3 03:44 kaggle.json


In [0]:
!cp -Rvf kaggle.json .kaggle/

'kaggle.json' -> '.kaggle/kaggle.json'


In [0]:
import json

token = {"username":"jimitogni","key":"aea778a52ae308a26e4ad5dad779ada9"}

with open('/content/.kaggle/kaggle.json', 'w') as file: json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [1]:
!kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


praveengovi/coronahack-chest-xraydataset                    CoronaHack -Chest X-Ray-Dataset

allen-institute-for-ai/CORD-19-research-challenge           COVID-19 Open Research Dataset Challenge (CORD-19) 

roche-data-science-coalition/uncover                        UNCOVER COVID-19 Challenge 

mariaren/covid19-healthy-diet-dataset                       COVID-19 Healthy Diet Dataset 

In [0]:
!kaggle datasets list -s c/dog-breed-identification

ref                                               title                                 size  lastUpdated          downloadCount  
------------------------------------------------  -----------------------------------  -----  -------------------  -------------  
selfishgene/youtube-faces-with-facial-keypoints   YouTube Faces With Facial Keypoints   10GB  2017-11-14 05:46:08           4037  
dansbecker/dog-breed-identification               dog breed identification             834KB  2019-03-18 15:54:45            509  
devdgohil/the-oxfordiiit-pet-dataset              The Oxford-IIIT Pet Dataset          780MB  2019-06-14 20:03:38            371  
prashantkashyap/dog-breed-identification-dataset  Dog_breed_identification_dataset        0B  2017-11-21 02:52:16            232  


In [0]:
!ls -lha '/content/drive/My Drive/UNICAMP/databases/'

In [0]:
!unzip '/content/drive/My Drive/UNICAMP/databases/dog-breed-identification.zip' -d '/content/drive/My Drive/UNICAMP/databases/dog_breed'

In [0]:
!pwd

/content


In [0]:
%%shell

export KAGGLE_USERNAME=jimitogni
export KAGGLE_KEY=6ba3f466a063e70addd26514bb2cf41d

kaggle competitions download -d dog-breed-identification

In [0]:
!unzip /content/drive/My\ Drive/UNICAMP/databases/dog_breed/sample_submission.csv.zip

In [0]:
ls -lha /content/drive/My\ Drive/UNICAMP/databases/

In [0]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

In [0]:
#verificar essa merda inteira
def build_model(size, num_classes):
  inputs = Input((size, size, 3))
  backbone = MobileNetV2(input_tensor=inputs, include_top=False, weights="imagenet")
  backbone.trainable = False
  x = backbone.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation="relu")(x)
  x = Dense(1024, activation="relu")(x)
  x = Dense(512, activation="relu")(x)
  x = Dropout(0,2)(x)
  x = Dense(num_classes, activation='softmax')(x)

  model = tf.keras.Model(inputs, x)

  return model

In [0]:
def read_image(path, size):
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  image = cv2.resize(image, (size, size))
  image = image / 255.0
  image = image.astype(np.float32)
  
  return image

In [0]:
def parse_data(x, y):
  x = x.decode()

  num_class = 120
  size = 224

  image = read_image(x, size)
  label = [0] * num_classes
  label[y] = 1
  label = np.array(label)
  label = label.astype(np.int32)

  return image, label

In [0]:
def tf_dataset(x, y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()

  return dataset

In [0]:
def tf_parse(x, y):
  x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
  x.set_shape((224, 224, 3))
  y.set_shape((120))

  return x, y

In [0]:
path = "/content/drive/My Drive/UNICAMP/databases/dog_breed/"
train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*")
labels_path = ("/content/drive/My Drive/UNICAMP/databases/dog_breed/labels.csv")

In [158]:
labels_df = pd.read_csv(labels_path)
breed = labels_df["breed"].unique()
print("Numero de raças: ", len(breed))

Numero de raças:  120


In [0]:
#dicionario, nome e raça
breed2id = {name: i for i, name in enumerate(breed)}
id2breed = {i: name for i, name in enumerate(breed)}

ids = glob(train_path)
labels = []

for image_id in ids:
  image_id = image_id.split("/")[-1].split(".")[0]
  breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
  breed_idx = breed2id[breed_name]
  #print(image_id, breed_name)
  labels.append(breed_idx)
  #labels = list(ids)

ids = ids[:1000]
labels = labels[:1000]

In [0]:
#split data set
train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)

In [0]:
size = 224
num_classes = 120
lr = 0.0001
batch = 32
epochs = 200

In [162]:
#model
model = build_model(size, num_classes)#verificar
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=["acc"])
model.summary()

TypeError: ignored

In [0]:
#datasets
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

In [150]:
#training
checkpoint = ModelCheckpoint('best_model_dogs.h5',
                             monitor='val_loss',
                             mode=min,
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,
                          restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=5,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.0001)

callbacks = [checkpoint, earlystop, learning_rate_reduction]

train_steps = (len(train_x)//batch)
valid_steps = (len(valid_x)//batch)

model.fit(train_dataset,
          steps_per_epoch=train_steps,
          validation_steps=valid_steps,
          validation_data=valid_dataset,
          epochs=epochs,
          callbacks=callbacks)

Epoch 1/200
25/25 [==============================] - ETA: 0s - loss: 0.0032 - acc: 1.0000
Epoch 00001: val_loss improved from inf to 2.49109, saving model to best_model_dogs.h5
25/25 [==============================] - 10s 390ms/step - loss: 0.0032 - acc: 1.0000 - val_loss: 2.4911 - val_acc: 0.4167 - lr: 1.0000e-04
Epoch 2/200
25/25 [==============================] - ETA: 0s - loss: 0.0038 - acc: 1.0000
Epoch 00002: val_loss improved from 2.49109 to 2.48441, saving model to best_model_dogs.h5
25/25 [==============================] - 10s 386ms/step - loss: 0.0038 - acc: 1.0000 - val_loss: 2.4844 - val_acc: 0.4115 - lr: 1.0000e-04
Epoch 3/200
 8/25 [========>.....................] - ETA: 4s - loss: 0.0042 - acc: 1.0000

KeyboardInterrupt: ignored

In [0]:
import matplotlib.pyplot as plt
print()
print("------ RESULTADOS ------")
print()
plt.figure(figsize=(13, 4))
plt.subplot(1, 2, 1)
plt.plot(model.history['accuracy'], label="AUC Treino")
plt.plot(model.history['val_accuracy'], label="AUC VALIDAÇÃO")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(model.history['loss'], label="loss validação")
plt.plot(model.history['val_loss'], label="AUC validação")
plt.legend()
plt.tight_layout()
plt.show()

print()
# Evaluate the model
#scores = history.history.evaluate_generator(test_generator)
#print("\n%s: %.2f%%" % (history.metrics_names[1], scores[1]*100))

print()
print("Com os valores:")
print("batch size: {} - Epocas: {}".format(batch_size, epochs))

In [0]:
## load best model
model = tf.keras.models.load_model("best_model.h5")

In [0]:
for i, path in tqdm(enumerate(valid_x[:100])):
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = id2breed[label_idx]

    ori_breed = id2breed[valid_y[i]]
    ori_image = cv2.imread(path, cv2.IMREAD_COLOR)

    ori_image = cv2.putText(ori_image, breed_name, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    ori_image = cv2.putText(ori_image, ori_breed, (0, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imwrite(f"save3/valid_{i}.png", ori_image)